# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)  # Unlimited columns

# Cleaning

In [77]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df = df.drop(columns = 'customerID')
df.TotalCharges = df.TotalCharges.replace(' ', 0.0)
df.TotalCharges = df.TotalCharges.astype(float)
df.tenure = df.tenure.astype(float)
df = pd.get_dummies(df)
df.TotalCharges = df.TotalCharges.replace(' ', 0.0)
df.TotalCharges = df.TotalCharges.astype(float)
df.tenure = df.tenure.astype(float)
df = pd.get_dummies(df)
df.shape

(7043, 47)

In [88]:
y = df.Churn_Yes.values
X = df.drop(columns = ['Churn_No','Churn_Yes']).values

# Baseline multi-level perceptron

In [95]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

/anaconda3/envs/sandbox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [102]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=45, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
batch_size = [20]
epochs = [20]
param_grid = dict(batch_size=batch_size, epochs=epochs)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
print()
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Epoch 1/20
4695/4695 [==============================] - 4s 762us/step - loss: 8.5004 - acc: 0.4072
Epoch 2/20
4695/4695 [==============================] - 4s 952us/step - loss: 2.3790 - acc: 0.7625
Epoch 3/20
4695/4695 [==============================] - 5s 1ms/step - loss: 1.9544 - acc: 0.7681
Epoch 4/20
4695/4695 [==============================] - 3s 644us/step - loss: 1.5513 - acc: 0.7661
Epoch 5/20
4695/4695 [==============================] - 3s 592us/step - loss: 1.6666 - acc: 0.7544
Epoch 6/20
4695/4695 [==============================] - 3s 619us/step - loss: 1.5981 - acc: 0.7610
Epoch 7/20
4695/4695 [==============================] - 2s 374us/step - loss: 1.3938 - acc: 0.7602
Epoch 8/20
4695/4695 [==============================] - 2s 448us/step - loss: 1.3771 - acc: 0.7610
Epoch 9/20
4695/4695 [==============================] - 3s 703us/step - loss: 1.0235 - acc: 0.7646
Epoch 10/20
4695/4695 [==============================] - 4s 794us/step - loss: 1.2403 - acc: 0.7649 1s - loss: 

7043/7043 [==============================] - 9s 1ms/step - loss: 0.8604 - acc: 0.7718
Epoch 18/20
7043/7043 [==============================] - 5s 739us/step - loss: 0.6181 - acc: 0.7775
Epoch 19/20
7043/7043 [==============================] - 6s 805us/step - loss: 0.7961 - acc: 0.7687
Epoch 20/20
7043/7043 [==============================] - 7s 969us/step - loss: 0.6452 - acc: 0.7730
Best: 0.7983813721162427 using {'batch_size': 20, 'epochs': 20}

Means: 0.7983813721162427, Stdev: 0.006833770971778664 with: {'batch_size': 20, 'epochs': 20}


In [105]:
# Report Results
print(f"Best: {grid_result.best_score_:.3f} using {grid_result.best_params_}")
print()
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean:.3f}, Stdev: {stdev:.3f} with: {param}") 

Best: 0.798 using {'batch_size': 20, 'epochs': 20}

Means: 0.798, Stdev: 0.007 with: {'batch_size': 20, 'epochs': 20}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?